# Import Libraries

In [47]:
import numpy as np
import pandas as pd

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import TfidfModel

# from gensim.models import CoherenceModel

# import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim

Notebook made using this video:  
https://www.youtube.com/watch?v=TKjjlp5_r7o

Another one to watch later:  
https://www.youtube.com/watch?v=UEn3xHNBXJU

# Read and process data

In [7]:
# read datafile
df = pd.read_csv('../project_data/complaints_processed.csv')

# clean up by removing NaN's, etc.
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df = df.dropna()
df.drop(df.loc[df['narrative'] == 'name'].index, inplace=True)

# Trying out on portion of dataframe

In [8]:
# just getting the texts from the first 1000 lines
partial_df = df['narrative'][0:1000]

In [9]:
partial_df.head()

0    purchase order day shipping amount receive pro...
1    forwarded message date tue subject please inve...
2    forwarded message cc sent friday pdt subject f...
3    payment history missing credit report speciali...
4    payment history missing credit report made mis...
Name: narrative, dtype: object

Testing to split strings. Gensim requires bag of words data (list of lists of individual strings)

In [10]:
partial_df[0].split()[0:5]

['purchase', 'order', 'day', 'shipping', 'amount']

Make the datataframe into a list of individual words

In [11]:
data_words = partial_df.tolist()

for i in range(len(data_words)):
    data_words[i] = data_words[i].split()

In [40]:
id2word = corpora.Dictionary(data_words)

In [13]:
print(id2word) 

Dictionary(5396 unique tokens: ['accordance', 'address', 'adjustment', 'adjustmentmerchandiserobert', 'agreed']...)


`id2word` is a dictionary with each token given an id.

`id2word.token2id` -->

`{'accordance': 0,
 'address': 1,
 'adjustment': 2,
 'adjustmentmerchandiserobert': 3,
 'agreed': 4,
 'although': 5, ...}`

In [9]:
# id2word.token2id

In [14]:
id2word.doc2bow(['address', 'adjustment', 'cancel'])

[(1, 1), (2, 1), (16, 1)]

`doc2bow` creates a tuple of (token_id, token_count). Here I'm making a list of these tuples for the gensim

In [15]:
corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

In [16]:
# the first five tuples of the first complaint
corpus[0][0:5]

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)]

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [18]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')

In [19]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.155212  0.343767       1        1  56.942843
4     -0.071174 -0.246817       2        1  25.559394
0     -0.250372 -0.048176       3        1  11.222665
3      0.329666 -0.187052       4        1   3.280452
1      0.147092  0.138278       5        1   2.994646, topic_info=            Term          Freq         Total Category  logprob  loglift
235     consumer  11869.000000  11869.000000  Default  30.0000  30.0000
1549    identity   2862.000000   2862.000000  Default  29.0000  29.0000
350       report   2886.000000   2886.000000  Default  28.0000  28.0000
1550       theft   2531.000000   2531.000000  Default  27.0000  27.0000
144      account   2654.000000   2654.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
471   inaccurate     29.175588     98.703761   Topic5  -5.2757   2.2896
336         made     32.664357    252.750342   Topic5  -5.1628   1.4622
1809     failure     29.900878    158.080441   Topic5  -5.2512   1.8431
27          date     34.484720    858.437314   Topic5  -5.1086   0.2937
788       result     31.976262    884.799595   Topic5  -5.1841   0.1880

[327 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
312       2  0.989983             able
367       4  0.951868            abuse
0         1  0.997497       accordance
144       1  0.257325          account
144       2  0.288219          account
...     ...       ...              ...
2648      5  0.806842            worst
218       2  0.997607            would
4192      5  0.930777    xxxxapartment
4193      5  0.929280  xxxxmultifamily
4194      5  0.849091       xxxxsingle

[487 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 5, 1, 4, 2])

## Repeat the above with TF-IDF

In [38]:
# for i in range(0, len(corpus)):
#     bow = corpus[i]
#     tfidf_ids = [id for id, value in tfidf[bow]]
#     bow_ids = [id for id, value in bow]
#     low_value_words = []
#     low_value_words = [id for id, value in tfidf[bow] if value < low_value]
#     drops = low_value_words + words_missing_in_tfidf
#     for item in drops:
#         words.append(id2word[item])
#     words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]
    
#     new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
#     corpus[i] = new_bow

# This one says don't use LDA with TF-IDF 
# https://datascience.stackexchange.com/questions/21950/why-we-should-not-feed-lda-with-tfidf

In [33]:
# tfidf = TfidfModel(corpus, id2word=id2word)

# low_value = 0.03
# low_value_words = []

# for bow in corpus:
#     low_value_words += [id for id, value in tfidf[bow] if value < low_value]
    
# id2word.filter_tokens(bad_ids=low_value_words)

In [41]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=10,
                                            random_state=100, 
                                            update_every=1,
                                            chunksize=100,
                                            passes=5,
                                            alpha="auto")

# Using whole dataframe

Processing dataframe into list of lists of words

In [42]:
data_words = df['narrative'].tolist()

for i in range(len(data_words)):
    data_words[i] = data_words[i].split()

Prepping the data for `lda_model`

In [43]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

In [18]:
# from gensim.models import TfidfModel

# tfidf = TfidfModel(corpus, id2word=id2word)

# low_value = 0.03
# low_value_words = []

# for bow in corpus:
#     low_value_words += [id for id, value in tfidf[bow] if value < low_value]
    
# id2word.filter_tokens(bad_ids=low_value_words)

# # not sure what this does exactly, but I had an index out of bounds error when I ran the model. 
# # Googling around, I found this might help:
# id2word.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)


Creating the model

In [44]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=5,
                                           alpha="auto")

Visualizing the model

In [45]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')

C:\Users\halpe\.conda\envs\learn-env\lib\site-packages\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
C:\Users\halpe\.conda\envs\learn-env\lib\site-packages\joblib\numpy_pickle.py:104: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


In [46]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.194152  0.183555       1        1  34.138275
4      0.136820  0.422214       2        1  14.423668
8     -0.076519  0.041101       3        1  14.048493
2     -0.147749  0.377987       4        1  13.073616
0      0.073011 -0.437552       5        1   7.875431
5     -0.424388  0.176786       6        1   5.593144
1     -0.203710 -0.326923       7        1   5.397785
9     -0.422251 -0.167416       8        1   4.221614
3      0.366237 -0.296434       9        1   0.697557
7      0.504397  0.026682      10        1   0.530417, topic_info=                Term           Freq          Total Category  logprob  loglift
144          account  260377.000000  260377.000000  Default  30.0000  30.0000
162           credit  313997.000000  313997.000000  Default  29.0000  29.0000
350           report  179547.000000  179547.000000  Default  28.0000  28.0000
404      information  169299.000000  169299.000000  Default  27.0000  27.0000
193          payment  127124.000000  127124.000000  Default  26.0000  26.0000
...              ...            ...            ...      ...      ...      ...
2361          region     337.751491     338.608989  Topic10  -5.3212   5.2367
2312          market     320.045987     320.903411  Topic10  -5.3751   5.2366
5239  securitization    1491.886309    1504.278841  Topic10  -3.8358   5.2310
3443           reptg     667.187512     700.084742  Topic10  -4.6405   5.1911
896         transfer    5837.204350   16932.310865  Topic10  -2.4715   4.1743

[540 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2358      8  0.999597       acc
9608     10  0.997346   accepts
1078      3  0.983620    access
1078      8  0.016218    access
144       5  0.999998   account
...     ...       ...       ...
777       2  0.268622  xxxxxxxx
777       5  0.731288  xxxxxxxx
311       1  0.608447      year
311       4  0.361848      year
311       5  0.029691      year

[835 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 5, 9, 3, 1, 6, 2, 10, 4, 8])

Later on, you may want to try TFIDF

In [22]:
# from gensim.models import TfidfModel

# tfidf = TfidfModel(corpus, id2word=id2word)

# low_value = 0.03

# words = []
# words_missing_in_tfidf = []

In [23]:
# vectorizer = TfidfVectorizer(ngram_range=(1,1))
# X = vectorizer.fit_transform(partial_df['narrative'])
# feature_names = vectorizer.get_feature_names()